 ---
 ---
   **Problem Formulation** 

 ---
 ---

*  **The input:** 
 *  Training dat (data) : it containd the features and the label data.
 *  Testing data (test) : it contains the features.

* **the output**
 * it will be the label data (the raiting column) 

*  **Function used**  
 *   classification

 *   predection

*   **Challenges**

 *   The dataset is not clean, and we need some preprocessing depending on the models we choice,
 
 - changing the multipul  hyperparameter to give a good accuracy/f1 score

*    **the imapct** 
 *   the impact that the model will fits will on the training data and predect well on the test data or any data that we enter, And that will have a huge impact in real life if that uesd we will be able to predect right and proratize what is important.
 
 - we are going to be able to evaluate our mdels correctly and we can help the customer to find what they really want. 

*    **ideal solution**
 *   the ideal solutition happens when our model is able to classify and predict very well on any data the model gets , And also that happens when our data is cleaned perfectly  and 

 - it happens when choose the perfecrt model who can deals with type of dat and not to forget the gyperparameter tuning .

----

**the experimental protocol used is 'Hold out method'**

The following is the process of using the hold-out method for model evaluation:

- Split the dataset into two parts 

- Train the model on the training dataset; While training the model, some fixed set of hyper parameters is selected.

- Test or evaluate the model on the held-out test dataset
Train the final model on the entire dataset to get a model which can generalize better on the unseen or future dataset. 
---


 







##**Importing libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()
from pathlib import Path

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit

##Read the data

In [ ]:
# Read the data
data = pd.read_csv('/content/drive/MyDrive/DM_assignment_3/xy_train.csv', sep=",", na_values=[""])
test = pd.read_csv('/content/drive/MyDrive/DM_assignment_3/x_test.csv', sep=",", na_values=[""])

##**Over view of the data**

---
**the over view always help us take a better look over tge data and help us realize anything needed to be fixed sometimes**.
**As we can see that there were the id column and we needed to drop it from the training and test data**. 

----

In [ ]:
data

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
test

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House...
59148,59148,Napoleon returns from his exile on the island ...
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
# save tge id for later use we we writting the output in the file
id_ = data['id']
id_test = test['id']

##Dropping the id

---
`if we didn't drop the id it will cause a miss leading in the classification.`

---

In [ ]:
#drop the id from the trainig data
data.drop(['id'],axis=1,inplace=True)

In [ ]:
#drop the id from the test data
test.drop(['id'],axis=1,inplace=True)

In [ ]:
data

,text,label
0,A group of friends began to volunteer at a hom...,0
1,British Prime Minister @Theresa_May on Nerve A...,0
2,"In 1961, Goodyear released a kit that allows P...",0
3,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...
59995,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
test

,text
0,stargazer
1,yeah
2,PD: Phoenix car thief gets instructions from Y...
3,"As Trump Accuses Iran, He Has One Problem: His..."
4,"""Believers"" - Hezbollah 2011"
...,...
59146,Bicycle taxi drivers of New Delhi
59147,Trump blows up GOP's formula for winning House...
59148,Napoleon returns from his exile on the island ...
59149,Deep down he always wanted to be a ballet dancer


##**pre-preocessing**

####**text processing**

----

**We tried different techniques to clean the data**:
- I am using feature selection ***Bag of words***
- Removing the leter s at the end of every verb to get back to the infentive (not always right)

- Keep only ASCII + European Chars and whitespace, no digits.
- remove any html tags.
- remove single letter chars
- convert the upper case to lower case

----

 

In [ ]:
#nltk is the res[posable for importting the nlp functions and library
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))

#text processing 

def clean_text(text):
   
    #convert all whitespaces (tabs etc.) to single wspace and remove the s from the verb 
    RE_WSPACE = re.compile(r"http\s+", re.IGNORECASE)

    #remove any html tags (< /br> often found)
    RE_TAGS = re.compile(r"<[^>]+>")

    #Keep only ASCII + European Chars and whitespace, no digits
    RE_ASCII = re.compile(r"[^A-Za-zA-z ]", re.IGNORECASE)

    #remove single letter chars
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zA-z]\b", re.IGNORECASE)
    


    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    # we tokenize the data (split the data into words/toknizer)
    word_tokens = word_tokenize(text)

    # keep the lower case as it is 
    words_tokens_lower = [word.lower() for word in word_tokens]

    # # convert the upper case to lower case
    # words_filtered = [stemmer.stem(word) for word in words_tokens_lower if word not in stop_words]
    # # 
    text_clean = " ".join(words_tokens_lower)

    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# data['text_clean']=data['text'].apply(clean_text)

In [ ]:
# test['text_clean']=test['text'].apply(clean_text)

In [ ]:
#delete the words that it's lenght is less than zero in the training data
# Clean Comments
data["text_clean"] = data.loc[data["text"].str.len() > 0, "text"]

data["text_clean"] = data["text"].map(
    lambda x: clean_text(x) if isinstance(x, str) else x
)

In [ ]:
#delete the words that it's lenght is less than zero in the test data
# Clean Comments
test["text_clean"] = test.loc[test["text"].str.len() > 0, "text"]
test["text_clean"] = test["text"].map(
    lambda x: clean_text(x) if isinstance(x, str) else x
)

In [ ]:
data.head(3)

,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friends began to volunteer at homeles...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa_may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows ps to be ...


In [ ]:
test

,text,text_clean
0,stargazer,stargazer
1,yeah,yeah
2,PD: Phoenix car thief gets instructions from Y...,pd phoenix car thief gets instructions from yo...
3,"As Trump Accuses Iran, He Has One Problem: His...",as trump accuses iran he has one problem his o...
4,"""Believers"" - Hezbollah 2011",believers hezbollah
...,...,...
59146,Bicycle taxi drivers of New Delhi,bicycle taxi drivers of new delhi
59147,Trump blows up GOP's formula for winning House...,trump blows up gop formula for winning house r...
59148,Napoleon returns from his exile on the island ...,napoleon returns from his exile on the island ...
59149,Deep down he always wanted to be a ballet dancer,deep down he always wanted to be ballet dancer


In [ ]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data["text_clean"]).split()).value_counts()
word_freq[1:40]

to       30592
of       29236
in       23623
and      19815
for      12358
it       11395
on       11267
this     10359
is       10039
with      8518
from      7510
my        7309
that      7058
you       6544
his       6413
at        6363
by        5467
was       5462
he        5248
after     5142
an        4557
has       4212
as        4190
they      3959
are       3796
be        3484
out       3415
have      3413
[         3262
]         3241
one       3165
her       3127
new       3010
but       2974
who       2913
like      2912
up        2856
their     2852
not       2787
dtype: int64

In [ ]:
# Word Frequency of most common words (test)
word_freq_test = pd.Series(" ".join(test["text_clean"]).split()).value_counts()
word_freq_test[1:40]

of          9283
in          8732
to          8658
this        7973
on          4718
my          4676
and         4412
psbattle    4396
for         4037
is          3585
it          3423
with        3076
at          2727
you         2627
from        2605
that        2078
his         1910
like        1781
by          1547
was         1527
has         1515
man         1485
an          1480
after       1418
he          1398
out         1233
me          1230
new         1200
one         1164
[           1114
up          1111
not         1108
dog         1102
looks       1101
]           1086
as          1086
found       1079
are         1075
be          1064
dtype: int64

In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,renegotiated,1
1,veafo,1
2,aecf,1
3,scute,1
4,gular,1
5,febba,1
6,ctrap,1
7,aurka,1
8,marut,1
9,wahre,1


####Delet outliars from the label

After i res the label data i realized that i got three label and there is a huge difference between the classes so the the third class is 

In [ ]:
# Distribution of ratings
data["label"].value_counts(normalize=True)

0    0.536200
1    0.459933
2    0.003867
Name: label, dtype: float64

In [ ]:
#grtting the label that less than 2
data = data[data['label'] != 2 ]

In [ ]:
# Distribution of ratings
data["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

#### splitting trainig data

In [ ]:
#splitting the data to feature and label
X=data["text_clean"]
y=data["label"]

In [ ]:
# models to test
classifiers = [
    LogisticRegression(solver="sag", random_state=1),
    LinearSVC(random_state=1),
    RandomForestClassifier(random_state=1),
    XGBClassifier(random_state=1),
    MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ),
]
# get names of the objects in list (too lazy for c&p...)
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
print(f"Classifiers to test: {names}")

Classifiers to test: ['LogisticRegression', 'LinearSVC', 'RandomForestClassifier', 'XGBClassifier', 'MLPClassifier']


---
##**Trial_1**

---

---
###**Logistic Regression**

----

**In this trail im using** : 
- **Classifier** : Logistic Regression 
- **Hyperparameter tuning**:  GridSearch with Validation set 
- **analysier**: word  

----
Logistic regression is not a classifier. It is a probability/risk estimator, it allows for and expects "close calls". It will lead to optimum decision making because it does not try to trick the predictive signal into incorporating a utility function that is implicit whenever you classify observations.


---
 ---
**My thoughts**

- since LR not in the top of the classifiers used in the text classification i think the model will give not too high roc_auc score

----
---

---- 
- **Here we have the pipeline it has the Vectorizer and the Classifier, and we using the Logistic Regression**.

- **After that we fit the data using the Pipline**.

- **We are using the validation set to get the number of cv** 

----
**the experimental protocol used is 'Hold out method'**

The following is the process of using the hold-out method for model evaluation:

- Split the dataset into two parts (preferably based on 70-30% split; However, the percentage split will vary)

- Train the model on the training dataset; While training the model, some fixed set of hyper parameters is selected.

- Test or evaluate the model on the held-out test dataset
Train the final model on the entire dataset to get a model which can generalize better on the unseen or future dataset. 
---

In [ ]:
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(norm='l2')), ('my_classifier', LogisticRegression())])
pipe.fit(X, y)


# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


**Hyperparameters used in Grid search** :

---
- **penalty** : the regulrization to control overfitting 

- **C** : is inverse of regularization, the larger the C, the smaller is regularization, means that your algo is more prone to overfit the data

- **n_jobs** : Number of jobs to run in paralle.

- **cv** : int, cross-validation generator or an iterable

- **verbose** : Controls the verbosity: the higher, the more messages.

-----
**Hyperparameters used in TfidfVectorizer** :

----

- **ngram_range**: The lower and upper boundary of the range of n-values for different n-grams to be extracted.and we gave it the range of (1,2),(1,3) i was trying different compination to see the best works for it and (1,2) was the best fit for it.

- **max_df** : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold. i put two values to see the best fit and 0.3 was the best fit.

- **min_df**: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.i gave it three values and i kept changing them but this results was the best.

- **analyzer**: Whether the feature should be made of word or character n-grams. here we triad the word n gran analyzer.


In [ ]:

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 30, 5),
    "tfidf__analyzer": ["word"],

    "my_classifier__penalty" : ['l2'],  
    'my_classifier__C' : [0.6], # 1.5 the best score for me 
    'my_classifier__solver': ['newton-cg']       
}
# it is quite slow so we do 4 for now
pipe_clf = GridSearchCV(pipe, params, n_jobs=2,cv=pds,verbose=1, scoring="roc_auc")

#fit the data using the best hyperparameters
pipe_clf.fit(X, y)
print(pipe_clf.best_score_,pipe_clf.best_params_)

Fitting 1 folds for each of 10 candidates, totalling 10 fits
0.8870358286194489 {'my_classifier__C': 0.6, 'my_classifier__penalty': 'l2', 'my_classifier__solver': 'newton-cg', 'tfidf__analyzer': 'word', 'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2)}


-----
----

- After performaing the model and compare with kagel scores i realized that the model perform very good.it gave me an accuracy = 0.887.
but when i uploaded in kagel it gave me 0.862 which is pretty good comparig to the other models.

- **"my thoughts"** 

 - i thought the model will give me a lower accuracy, according to the classifiers that works well with  text classification LR wasn't one of them.but from what we see the Logistc regrssion with the value of the previous hyperparameter works well together.

---
----

In [ ]:
#predect the the testing data
y_test_pred_ = pipe_clf.predict_proba(test["text_clean"])

In [ ]:
#Write the output in a file 
write_ = pd.DataFrame({'id': id_test,'label': y_test_pred_[:,1]})
write_.to_csv('/content/drive/MyDrive/DM_assignment_3/assignment2_Rand.csv', encoding='utf-8', index=False)

---

##**Trial_2**

---

---
###**Logistic Regression**
----

**In this trail im using** : 
- **Classifier**: Logistic Regression 
- **Hyperparameter tuning**: GridSearch with Validation set 
- **analysier**: char  


---
 ---
**My thoughts**

- After the first trial i got the realize that the Logistic regresiion clasiifier is performing well with this model so im going to try it again with different paramters like the hyperparameter tuning and the analysis, i think the model is going to give a lesser accuracy than the previous one but not so much and that's because of the analysier. 

----
---

**Hyperparameters used in Grid search** :

---
- **penalty** : the regulrization to control overfitting 

- **C** : is inverse of regularization, the larger the C, the smaller is regularization, means that your algo is more prone to overfit the data

- **n_jobs** : Number of jobs to run in paralle.

- **cv** : int, cross-validation generator or an iterable

- **verbose** : Controls the verbosity: the higher, the more messages.

-----
**Hyperparameters used in TfidfVectorizer** :

----

- **ngram_range**: The lower and upper boundary of the range of n-values for different n-grams to be extracted.in this trial i tried to change the range to different thong that was the best for this trial and we gave it the range of (1,2) was the best fit for it.

- **max_df** : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold.i put two values to see the best fit and 0.3 was the best fit.

- **min_df**: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.i gave it three values and i kept changing them but this results was the best.

- **analyzer**: Whether the feature should be made of word or character n-grams.here we triad the character n gran analyzer


In [ ]:

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 30, 5),
    "tfidf__analyzer": ["char"],

    "my_classifier__penalty" : ['l2'],  
    'my_classifier__C' : [0.6,0.7],  
    'my_classifier__solver': ['newton-cg']       
}

pipe_clf = GridSearchCV(pipe, params, n_jobs=2,cv=pds,verbose=1, scoring="roc_auc")

#fit the data with the hyperparameter tuning
pipe_clf.fit(X, y)
print(pipe_clf.best_score_,pipe_clf.best_params_)

Fitting 1 folds for each of 20 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.8506765566377816 {'my_classifier__C': 0.7, 'my_classifier__penalty': 'l2', 'my_classifier__solver': 'newton-cg', 'tfidf__analyzer': 'char', 'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}


-----
----

- After performaing the model and compare with kagel scores i realized that the model perform not too good.it gave me an accuracy = 0.850.
but when i uploaded in kagel it gave me 0.70 which is it's too bad.

- **"my thoughts"** 

 - i expected the model to give a lower accuracy because of the analyzer.

---
----

In [ ]:
#predect the the testing data
y_test_pred_ = pipe_clf.predict_proba(test["text_clean"])

In [ ]:
#Write the output in a file
write_ = pd.DataFrame({'id': id_test,'label': y_test_pred_[:,1]})
write_.to_csv('/content/drive/MyDrive/DM_assignment_3/assignment2_LR_char.csv', encoding='utf-8', index=False)

----
 
##**Trial_3**

---

----
###**XGBClassifier**

---

**In this trail im using** : 
- **Classifier**: XGBClassifier 
- **Hyperparameter tunin**g:  GridSearch with Validation set 
- **analysier**: word 



----
---
**My thoughts**

 It can help you to predict any kind of data if you have already predicted data before. You can classify any kind of data. It can be used for text classification too.. it's my first encoter to use this classifier in text classificatoion so i think from what i red that it will give a good accuracy .
___
---

----
- **Here we have the pipeline it has the Vectorizer and the Classifier i am using the XGBclassifier**.
- **After that we fit the data using the Pipline**.

- **We are using the validation set to get the nuber of cv** 

----

In [ ]:
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(norm='l2')), ('my_classifier', XGBClassifier())])
pipe.fit(X, y)

# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

--- 

**Hyperparameters used in Grid search** :

---
- **max_depth** : Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.i tried different values and i got to conclusion that thae calues im using (7) is the best value of all the values i used. 

- **gamma**: Minimum loss reduction required to make a further partition on a leaf node of the tree.the value i used was the best of all the valus i used before[0.5 ,1, 1.5]

- **n_estimators** : A object of that type is instantiated for each grid point. This is assumed to implement the scikit-learn estimator interface. Either estimator needs to provide a score function, or scoring must be passed.

- **n_jobs** : Number of jobs to run in paralle.

- **cv** : int, cross-validation generator or an iterable

- **verbose** : Controls the verbosity: the higher, the more messages.

-----
**Hyperparameters used in TfidfVectorizer** :

----

- **ngram_range**: The lower and upper boundary of the range of n-values for different n-grams to be extracted.

- **max_df** : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold

- **min_df**: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.

- **analyzer**: Whether the feature should be made of word or character n-grams



In [ ]:
# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 30, 5),
     "tfidf__analyzer": ["word"],

    "my_classifier__max_depth" : [6,7],
    'my_classifier__gamma' :[1],
    'my_classifier__learning_rate' :[0.05,0.01,0.1],
    'my_classifier__n_estimators':  [300]      
}

pipe_clf = GridSearchCV(pipe, params, n_jobs=2,cv=pds,verbose=1, scoring="roc_auc")

pipe_clf.fit(X, y)

print(pipe_clf.best_score_,pipe_clf.best_params_)

Fitting 1 folds for each of 60 candidates, totalling 60 fits
0.8612442740364465 {'my_classifier__gamma': 1, 'my_classifier__learning_rate': 0.1, 'my_classifier__max_depth': 7, 'my_classifier__n_estimators': 300, 'tfidf__analyzer': 'word', 'tfidf__max_df': 0.3, 'tfidf__min_df': 15, 'tfidf__ngram_range': (1, 3)}


-----
----
- After performaing the model and compare with kagel scores i realized that the model perform not not too good.it gave me an accuracy = 0.861.
but when i uploaded in kagel it gave me 0.83 which is it's too bad.

- **"my thoughts"** 

 - Well the model give not too bad accuracy comparing the other classifier.. i think the compination of hyperparameter didn't work well as expected. at the next trial im going to use the same classifier but with different hyperparamter. . 

---
---

In [ ]:
#predect the the testing data
y_test_pred_2 = pipe_clf.predict_proba(test["text_clean"])

In [ ]:
#Write the output in a file
write_ = pd.DataFrame({'id': id_test,'label': y_test_pred_2[:,1]})
write_.to_csv('/content/drive/MyDrive/DM_assignment_3/assignment2_xgbost.csv', encoding='utf-8', index=False)

---
##**Trial_4**

---

----
###**XGBClassifier**

---

**In this trail im using** : 
- Classifier: XGBClassifier 
- Hyperparameter tuning:  RandomizedSearchCV with Validation set 
- analysier: char  



--- 

**Hyperparameters used in Randomsearch** :

---
- **max_depth** : Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.

- **gamma**: Minimum loss reduction required to make a further partition on a leaf node of the tree.

- **n_estimators** : A object of that type is instantiated for each grid point. This is assumed to implement the scikit-learn estimator interface. Either estimator needs to provide a score function, or scoring must be passed.

- **n_jobs** : Number of jobs to run in paralle.

- **cv** : int, cross-validation generator or an iterable

- **verbose** : Controls the verbosity: the higher, the more messages.

-----
**Hyperparameters used in TfidfVectorizer** :

----

- **ngram_range**: The lower and upper boundary of the range of n-values for different n-grams to be extracted.

- **max_df** : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold

- **min_df**: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.

- **analyzer**: Whether the feature should be made of word or character n-grams



In [ ]:
# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 30, 5),
     "tfidf__analyzer": ["char"],

    "my_classifier__max_depth" : [6,5],
    'my_classifier__gamma' :[0.5, 1, 1.5],
    'my_classifier__learning_rate' :[0.05,0.001,],
    'my_classifier__n_estimators':  [400]      
}

pipe_clf = RandomizedSearchCV(pipe, params, n_jobs=2,cv=pds,verbose=1, scoring="roc_auc")

pipe_clf.fit(X, y)

print(pipe_clf.best_score_,pipe_clf.best_params_)

Fitting 1 folds for each of 10 candidates, totalling 10 fits
0.8513027609784268 {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 10, 'tfidf__max_df': 0.3, 'tfidf__analyzer': 'char', 'my_classifier__n_estimators': 400, 'my_classifier__max_depth': 6, 'my_classifier__learning_rate': 0.05, 'my_classifier__gamma': 1}


-----
----

- After performaing the model and compare with kagel scores i realized that the model perform not not too good.it gave me an accuracy = 0.85.
but when i uploaded in kagel it gave me 0.72 which is it's too bad.

- **"my thoughts"** 

 - As expected the model gives a lower accuracy because of the analyzer nt to bad it's a bit close to when the analyzer was word. 

---
----

In [ ]:
#predect the the testing data
y_test_pred_2 = pipe_clf.predict_proba(test["text_clean"])

In [ ]:
#Write the output in a file
write_ = pd.DataFrame({'id': id_test,'label': y_test_pred_2[:,1]})
write_.to_csv('/content/drive/MyDrive/DM_assignment_3/assignment2_xgbost_RS.csv', encoding='utf-8', index=False)

---
##**Trial_5**

---

----
####**Random forest**

---

**In this trail im using** : 
- **Classifier**: RamdomForest 
- **Hyperparameter tuning**:  GridSearchCV with Validation set 
- **analysier**: word




---
---

**My thoughts**

- it is recommended to test several models regardless of their theoretical performance, because their accuracy is dependent of the training dataset. i don't know how it woll go with this but let's find out.    

 -----
 ----

----
- **Here we have the pipeline it has the Vectorizer and the Classifier i am using the RandomForest**.
- **After that we fit the data using the Pipline**.

- **We are using the validation set to get the nuber of cv** 

----

In [ ]:
# feature creation THE PIPLINE
pipe = Pipeline([("tfidf", TfidfVectorizer(norm='l2')), ('my_classifier', RandomForestClassifier())])
pipe.fit(X, y)

# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

--- 

**Hyperparameters used in Grid search** :

---
- **max_depth** : Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.

- **min_samples_leaf**: The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression

- **min_samples_split**: The minimum number of samples required to split an internal node

- **n_estimators** : A object of that type is instantiated for each grid point. This is assumed to implement the scikit-learn estimator interface. Either estimator needs to provide a score function, or scoring must be passed.

- **n_jobs** : Number of jobs to run in paralle.

- **cv** : int, cross-validation generator or an iterable

- **verbose** : Controls the verbosity: the higher, the more messages.

-----
**Hyperparameters used in TfidfVectorizer** :

----

- **ngram_range**: The lower and upper boundary of the range of n-values for different n-grams to be extracted.

- **max_df** : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold

- **min_df**: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.

- **analyzer**: Whether the feature should be made of word or character n-grams



In [ ]:

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 30, 5),
    "tfidf__analyzer": ["word"],

    'my_classifier__max_depth': [50, 60],
    'my_classifier__min_samples_leaf': [4],
    'my_classifier__min_samples_split': [5, 10],
    'my_classifier__n_estimators': [200]      
}

pipe_clf = GridSearchCV(pipe, params, n_jobs=2,cv=pds,verbose=1, scoring="roc_auc")

# fithe the data using the hyperparameter tuning 
pipe_clf.fit(X, y)

print(pipe_clf.best_score_,pipe_clf.best_params_)

Fitting 1 folds for each of 40 candidates, totalling 40 fits
0.847952013761037 {'my_classifier__max_depth': 60, 'my_classifier__min_samples_leaf': 4, 'my_classifier__min_samples_split': 5, 'my_classifier__n_estimators': 200, 'tfidf__analyzer': 'word', 'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2)}


-----
----

- After performing we realize that the model perform not too bad comparing to other models.we after triaing diferent compination that was the best that get me this accuracy maybe trying different comoination will make difference in next trial. 

---
----

In [ ]:
#predect the the test file
y_test_pred_3 = pipe_clf.predict_proba(test["text_clean"])

In [ ]:
#Write the the predected value into file
write_ = pd.DataFrame({'id': id_test,'label': y_test_pred_3[:,1]})
write_.to_csv('/content/drive/MyDrive/DM_assignment_3/assignment2__RF_GS.csv', encoding='utf-8', index=False)

---
##**Trial_6**

---

----
###**Random forest**

---

**In this trail im using** : 
- **Classifier**: RandomForst 
- **Hyperparameter tuning**:  RandomizedSearchCV with Validation set 
- **analysier**: char  


--- 

**Hyperparameters used in Grid search** :

---
- **max_depth** : Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.

- **min_samples_leaf**: The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression

- **min_samples_split**: The minimum number of samples required to split an internal node

- **n_estimators** : A object of that type is instantiated for each grid point. This is assumed to implement the scikit-learn estimator interface. Either estimator needs to provide a score function, or scoring must be passed.

- **n_jobs** : Number of jobs to run in paralle.

- **cv** : int, cross-validation generator or an iterable

- **verbose** : Controls the verbosity: the higher, the more messages.

-----
**Hyperparameters used in TfidfVectorizer** :

----

- **ngram_range**: The lower and upper boundary of the range of n-values for different n-grams to be extracted.

- **max_df** : When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold

- **min_df**: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.

- **analyzer**: Whether the feature should be made of word or character n-grams



In [ ]:

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 30, 5),
    "tfidf__analyzer": ["char"],

    'my_classifier__max_depth': [30, 50, 60],
    'my_classifier__min_samples_leaf': [1, 2, 4],
    'my_classifier__min_samples_split': [5, 10],
    'my_classifier__n_estimators': [800]      
}

pipe_clf = RandomizedSearchCV(pipe, params, n_jobs=2,cv=pds,verbose=1, scoring="roc_auc")
#fit the data using the hupyparameter tunning
pipe_clf.fit(X, y)

print(pipe_clf.best_score_,pipe_clf.best_params_)

Fitting 1 folds for each of 10 candidates, totalling 10 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.8377471172905129 {'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 20, 'tfidf__max_df': 0.3, 'tfidf__analyzer': 'char', 'my_classifier__n_estimators': 800, 'my_classifier__min_samples_split': 10, 'my_classifier__min_samples_leaf': 1, 'my_classifier__max_depth': 60}


-----
----

- After performaing the model and compare with kagel scores i realized that the model perform not not too good.i will definatly try differnt compination next time.
---
----

In [ ]:
# predect the the test file 
y_test_pred_3 = pipe_clf.predict_proba(test["text_clean"])

In [ ]:
#wRITTING THE OUTPUT in a file
write_ = pd.DataFrame({'id': id_test,'label': y_test_pred_3[:,1]})
write_.to_csv('/content/drive/MyDrive/DM_assignment_3/assignment2_RF_RS.csv', encoding='utf-8', index=False)


#**Questions**


---
1-**What is the difference between Character n-gram and Word n-gram**? **Which one tends to suffer more from the OOV issue**?

---

**Word N-grams**
-  means a sequence of N words. in NLP n-grams are used for a variety of things. Some examples include auto completion of sentences (such as the one we see in Gmail these days), auto spell check (yes, we can do that as well), and to a certain extent, we can check for grammar in a given sentence. 

- The alignment systems are traditionally based on word
N-grams splitting. The observation of the morphological variety of languages, even inside a single linguistic family, quickly shows
that the word granularity is inadequate to provide a widely multilingual system, i.e. a language independent system able to handle
flexional languages as well as positional languages. Instead, when starting from a multilingual collection to focus on pairs of texts,
we defend that character N-grams alignment is more efficient than word N-grams alignment.


**Character N-grams**

- Character N-grams (of at least 3 characters) that are common to words meaning “transport” in the same texts sample in French, Spanish and Greek and their respective frequency.The idea of considering character n-grams rather than words has been used for author identification ,language identification, speech analysis, text categorization, numerical classification of multilingual documents and information retrieval. in most of the NLP applications, the number N of characters considered (N-grams) is defined before hand and constant. In general, the character N-grams considered are bigrams or trigrams (4-grams or
5-grams in the case of Mcnamee & Mayfield (2004). On the contrary, we consider character N-grams2 of various size:
we consider all repeated strings of our corpus (frequency equal or greater than 2). 

 **Word N-grams is the one suggering from out-of-vocabulary (OOV) words**
 


---
2- **What is the difference between stop word removal and stemming? Are these techniques language-dependent**?

----






**Stop word elimination and stemming are commonly used method in indexing**.

**Both stemming and stopword removal are pretty popular preprocessing techniques for text classification**.


**Stop words**

-  Stop words are high frequency words that have little semantic weight and are thus unlikely to help the retrieval process. Usual practice in IR is to drop them from index.

**Stemming** 
 
 - Stemming conflates morphological variants of words in its root or stem. It
frees user from worrying about the truncation and inflection while framing queries and helps in reducing index size. Stemming does help in improving the retrieval performance. Particularly, recall is expected to improve after stemming.
-----

- **We do not always remove the stop words. The removal of stop words is highly dependent on the task we are performing and the goal we want to achieve**.


- **The stemming or lemmatization of words is the most popular language-dependent approach to document retrieval**. **We use the set of stemmers implemented in the Snowball language** 

- **stop word and stemming is a word deppendant**

 ---
 **3**-**Is tokenization techniques language dependent? Why**?

 ----

- Tokenization is a way of separating a piece of text into smaller units called tokens. 

- **Yes** tokenization is language dependent.

- **As tokens are the building blocks of Natural Language, the most common way of processing the raw text happens at the token level**.**Tokenization is the foremost step while modeling text data. Tokenization is performed on the corpus to obtain tokens**.

- in english 

-----
4-**What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them**?

----

TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model can process only numerical data.


- **CountVectorizer**: Counts the frequency of all words in our corpus, sorts them and grabs the most recurring features (using max_features hyperparameter). But these results are mostly biased and our model might loose out on some of the important less frequent features.


- **TFIDFVectorizer**: TFIDF is a statistical measure said to have fixed the issues with CountVectorizer in some way. It consists of 2 parts, TF (Term Frequency) multiplied with IDF (Inverse Document Frequency). The main intuition being some words that appear frequently in 1 document and less frequently in other documents could be considered as providing extra insight for that 1 document and could help our model learn from this additional piece of information. In short, common words are penalized. These are relative frequencies identified as floating point numbers.

- **No** it's not feasible to use all of them one time , it cuz it differe from application to another and another.

-  an n-gram of size 1 is referred to as a "unigram"; size 2 is a "bigram"; size 3 is a "trigram". When N>3 this is usually referred to as four grams or five grams and so on.


Refreance

- https://towardsdatascience.com/text-pre-processing-stop-words-removal-using-different-libraries-f20bac19929a

- https://link.springer.com/content/pdf/10.1007/978-81-8489-203-1_31.pdf

- https://irlab.science.uva.nl/wp-content/papercite-data/pdf/kamps-language-dependent-2004.pdf

- https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/

- https://www.quora.com/How-are-TF-IDF-vectorizers-with-n-gram-features-created